## Alignment with BWA, Rebaler and BLAST

There are many algorithms to efficiently align short reads, although they are not optimal for long reads (e.g. ONT reads). To put it simple, long-reads are usually affected by structural variations and the indels that may be due to sequencing errors. For this reason, NanoDJ relies on aligners such as BWA, Rebaler or BLAST that are better for finding local matches. 

## Reference-based assembly with BWA

Burrows Wheeler Aligner ([BWA](https://github.com/lh3/bwa)) is carefully designed to achieve a good balance between performance and accuracy in the alignment. BWA is software package that includes tools for mapping ONT long reads to a reference, including many alternative alignment algorithms. Some of them are ideal for short-reads, while others are better suited for long reads. BWA-MEM is generally recommended for high-quality queries as it is faster and more accurate. BWA-MEM automatically chooses between local and end-to-end alignments, supports paired-end reads and performs chimeric alignment. The algorithm is robust to sequencing errors and chimeras, and is applicable to a wide range of sequence lengths from 70 bp to a few Mb. 

This algorithm is used many times in NanoDJ notebooks as a step for some of the applications. BWA needs the sequence reads and a reference as inputs and supports more than one execution thread with the **-t** option.

Before running BWA-MEM, the user will first need to index the reference genome (FASTA):

In [ ]:
bwa index data/sample/reference/NZ_CP010867.1_Ref.fasta

Once the reference is indexed, BWA-MEM can be run using the **-t** option to allow multithreaded execution. One must specify the (previously indexed) reference, the reads file (either as FASTA or FASTQ), and redirect the output to a file (SAM format):

In [ ]:
bwa mem -t 2 data/sample/reference/NZ_CP010867.1_Ref.fasta data/sample/reads.fastq > data/sample/bwa_output.sam

## Reference-based assembly with Rebaler

[Rebaler](https://github.com/rrwick/Rebaler) is used to obtain reference-based assemblies but can also reassemble/polish an assembly of long reads, using a reference assembly to guide the large-scale structure. Another advantage of Rebaler is that the reference assembly sequence does not influence the sequence of the resulting assembly.

After loading the reference, Rebaler uses minimap2 to align long reads to the reference. Then, it Removes lower quality alignments (judged by length, identity and size of indels) until the reference is just covered. Any given position in the reference should now have a coverage of 1 or 2 (or 0 if the reads failed to cover a spot). The reference sequence is replaced with corresponding read fragments to produce an unpolished assembly. If parts of the reference had no read coverage, the original reference sequence will be left in place.

Once the Rebaler assembly is built, multiple [Racon](https://github.com/isovic/racon) rounds are run to polish the consensus sequence.

In [ ]:
rebaler -h

In [ ]:
rebaler -t 48 data/sample/reference/NZ_CP010867.1_Ref.fasta data/sample/reads.fastq > data/sample/assembly_with_rebaler.fasta

## Reference-based assembly with BLAST

Basic Local Alignment Search Tool (BLAST) is a classic local sequence alignment, which compares nucleotide to sequence databases finding similarity regions between sequences. BLAST uses a heuristic algorithm. Therefore, there is no guarantee that BLAST finds the correct solution albeit it will calculate the significance of the results, proving a parameter to score the results obtained.

The BLAST algorithm has three main stages: training (finds local matches), extension (alignment is extended on both sides of the words) and evaluation (evaluate the statistical significance of the resulting alignments and eliminates the inconsistent ones). For this, BLAST needs a database where the reference sequences (all in one FASTA file) are indexed for comparison. BLAST has many different commands available (https://www.ncbi.nlm.nih.gov/books/NBK279684/). However, we will focus on a few basic ones that may be useful for the user:

| Option | Type | Description | Notes |
| :------: | :----: | :-----------: | :-----: |
| evalue | real | Expect value (E) for saving hits.(Default value = 10.0)| E-value: expected number of chance alignments; the smaller the E-value, the better the match. |
| html | flag | Produce HTML output| . |
| outfmt | string | Alignment pairwise| Can be additionally configured by indicating “length means” or “Pident means” to obtain the legth of alignments and the % of identical matches, respectively. For example: -outfmt 0 length means |

Before proceeding to the alignment step, the user first needs to create a local database with the reference sequences among which the alignment is to be done. This can be done with the following arguments:

In [2]:
makeblastdb -in data/sample/reference/NZ_CP010867.1_Ref.fasta -parse_seqids -dbtype nucl



Building a new DB, current time: 11/26/2018 21:55:10
New DB name:   /home/jovyan/notebooks/data/sample/reference/NZ_CP010867.1_Ref.fasta
New DB title:  data/sample/reference/NZ_CP010867.1_Ref.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.0301111 seconds.


Once the local database is created, the user is ready to query it. The input is a FASTA file with unmapped sequences. To query the database, the following arguments can be used:

In [3]:
mkdir data/sample/BLAST_Output
blastn -query data/sample/reads.fasta -db data/sample/reference/NZ_CP010867.1_Ref.fasta -task blastn -dust no -outfmt "10 qseqid evalue sseqid" -evalue 11 -max_hsps 1 -max_target_seqs 1 -num_threads 1 > data/sample/BLAST_Output/Sample.csv

The outputs are generated as csv files, which makes easier for pandas package to be processed and to plot the results.

### References

[1] Li H. and Durbin R. Fast and accurate short read alignment with Burrows-Wheeler Transform. Bioinformatics. 2009 Jul 15;25(14):1754-60. DOI: 10.1093/bioinformatics/btp324.

[2] Wick R. (2017). Rebaler: Reference-based long read assemblies of bacterial genomes. GitHub. URL https://github.com/rrwick/Rebaler

[3] Altschul S.F., Gish W., Miller W., Myers E.W., Lipman D.J. Basic local alignment search tool. J. Mol. Biol. Volume 215, Issue 3, 5 October 1990, Pages 403-410. DOI: https://doi.org/10.1016/S0022-2836(05)80360-2